In [23]:
import json
import os
from src.extractor import TextExtractor

template_path: str = os.path.join("src", "templates", "barclays_student_template.json")
pdf_path: str = os.path.join("data", "bank_statements", "barclays_student", "pdf", "barclays_student_march.pdf")
pdf_data_path: str = os.path.join("src", "pdf_data", "barclays_student_march_pdf_data.json")
output_path: str = os.path.join("src", "outputs", "barclays_student_march_output.json")

with open(pdf_path, "rb") as pdf_file:
    text_extractor = TextExtractor(pdf_file.read())
    extracted_data = text_extractor.extract_text()

    with open(pdf_data_path, "w") as f:
        json.dump(extracted_data, f)

template = json.load(open(template_path))
pdf_data = json.load(open(pdf_data_path))

output_data = json.load(open(output_path))


from src.parser import Parser, TableSplitter
from src.pdf_utils import ImageDrawer

parser = Parser()

page_number = 2

table_splitter = TableSplitter(template, parser)

page_content = pdf_data["pages"][page_number - 1]

template = json.load(open(template_path))

for rule in template["rules"][-1:]:
    if rule["type"] == "table":
        table_config = rule["config"]
        for column in table_config["columns"][:1]:
            coordinates = column["coordinates"]
            image_with_lines = ImageDrawer.draw_column_box_and_lines(pdf_path, table_splitter, page_content, coordinates, "line", page_number)
            image_with_lines.show()
            image_with_delimiters = ImageDrawer.draw_column_box_and_lines(pdf_path, table_splitter, page_content, coordinates, "delimiter", page_number)
            image_with_delimiters.show()

